In [1]:
from numba import jit
from numba import njit
import numba
import numpy as np
import math
import random
from helper_methods_cy import *
import matplotlib.pyplot as plt

In [13]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
@jit(nopython=True)
# @jit
def Get_mu(w_, a):

   # np.random.seed(2019)
    r = random.random()

    LEN = len(w_)
    

    cumsumw_ = 0
    
    for i in range(0,LEN):

        cumsumw_ += w_[i]
        
        if cumsumw_ >= r*a :
            
            return i

@jit(nopython=True)        
def Get_tau(a):

    # np.random.seed(2019)
    r = random.random()
    return (1.0/a) *math.log(1.0/r)

In [3]:
# @jit(nopython=True)
@jit
def Sim_run_helper(a,L,LEN,w_,rib_pos_,pos_jump_,pos_w_,Get_tau,Get_mu):
        
    enter = False
    do_exit = False
    move = False
    
#     a = sim_list_[0]
#     L = sim_list_[1]
#     LEN = sim_list_[2]
#     w_ = sim_list_[3]
#     rib_pos_ = sim_list_[4]
#     pos_jump_ = sim_list_[5]
#     pos_w_ = sim_list_[6]
        
        
    tau = Get_tau(a)
        
#     t += tau
    
    index = int(Get_mu(pos_w_,a))
        
    
#     if steady_reached:
            
#         sim.__calcDensity(tau)
    
        
    if pos_jump_[index] == 0:
        
        rib_pos_ = np.insert(rib_pos_,0,2)
        a -= w_[0]
            
        if len(rib_pos_) > 1:
            if rib_pos_[1] > L+2: 
                pos_jump_[index] = 2
                pos_w_[index] = w_[2]
                a += w_[2]
            elif rib_pos_[1] <= L+2:
                pos_jump_= np.delete(pos_jump_, index)
                pos_w_= np.delete(pos_w_, index)
                    
        else:
            pos_jump_[index] = 2
            pos_w_[index] = w_[2]
            a += w_[2]
                
            enter = True
        
    elif pos_jump_[index] == LEN:
        
        rib_pos_ = np.delete(rib_pos_, -1)
        a -= w_[-1]
            
        if len(rib_pos_) >= 1 and rib_pos_[-1] == LEN-L:
            pos_jump_[index] = rib_pos_[-1]
            pos_w_[index] = w_[rib_pos_[-1]]
            a += w_[rib_pos_[-1]]
            
        else:
            pos_jump_= np.delete(pos_jump_, index)
            pos_w_= np.delete(pos_w_, index)
                
        do_exit = True
        
    else:
        
        rib_pos_index = np.where(rib_pos_ == pos_jump_[index])
        rib_pos_index = rib_pos_index[0][0]
        rib_pos_[rib_pos_index] += int(1)
        a -= pos_w_[index]
            
            
        if len(rib_pos_) > rib_pos_index+1:
            if rib_pos_[rib_pos_index+1] - rib_pos_[rib_pos_index] > L:
                pos_jump_[index] = rib_pos_[rib_pos_index]
                pos_w_[index] = w_[rib_pos_[rib_pos_index]]
                a += w_[rib_pos_[rib_pos_index]]
            else:
                pos_jump_= np.delete(pos_jump_, index)
                pos_w_= np.delete(pos_w_, index)
        else:
                
            pos_jump_[index] = rib_pos_[rib_pos_index]
            pos_w_[index] = w_[rib_pos_[rib_pos_index]]
            a += w_[rib_pos_[rib_pos_index]]
                
        if rib_pos_index != 0 and rib_pos_[rib_pos_index] - rib_pos_[rib_pos_index-1] == L+1 :
                
            pos_jump_ = np.insert(pos_jump_,index,rib_pos_[rib_pos_index-1]) 
            pos_w_ = np.insert(pos_w_,index,w_[rib_pos_[rib_pos_index-1]])
            a += w_[rib_pos_[rib_pos_index-1]]
                
        elif rib_pos_index == 0 and rib_pos_[rib_pos_index] == L+2:
            pos_jump_ = np.insert(pos_jump_,0,0) 
            pos_w_ = np.insert(pos_w_,0,w_[0])
            a += w_[0]
                
        move = True
    
#     sim_list_ = [a,rib_pos_,pos_jump_,pos_w_]
 
    return a,rib_pos_,pos_jump_,pos_w_, tau, enter, do_exit, move

In [4]:
class Simulation:
    
    """
    Class containing the main methods to run the simulation
    """
    
    def __init__(self,L, rel_tol = 0.1, sweeps = 0):
        
        """
        Class initiator
        
        Inputs:
                L <int>: Size of ribosome
                rel_tol <float>: Maximum Relative current difference to determine steady state
                sweeps <int>: Number of sweeps to the the simulation for. (Both to reach
                                steady state and after)
                                
        """
        
        self.L = L
        self.sweeps = sweeps
        self.rel_tol = rel_tol
        
        
        
    def ExtractRates(self,file_name,alpha,beta):
        
         
        """
        Extract hopping rates from the file.
        
        Inputs:
                file_name <string>: Path/Name of file containing the rates
                alpha <float>: Inisitiation Rate
                beta <float>: Exit rate
                                
        """
    
        self.w_ = Extract_hopping_rates(open(file_name, 'r'), alpha,beta)
        self.LEN = len(self.w_) - 1
        self.RUNS = self.sweeps * self.LEN
        
    def ChangeAlpha(self,alpha):
        """
        Change the inisitiation rate
        
        Inputs:
                alpha <float>: Inisitiation Rate
                                
        """
        
        self.w_[0] = alpha
        
        return
    
    @jit
    def Reach_steady(self):
        
        """
        Reach the steady state for the simulation
                                
        """
        
        # Initialize lists needed for simulation
        
        self.pos_jump_ = np.zeros(1,dtype=int) # List of ribisome positions that can possible jump in next step
        self.pos_w_ = np.array([self.w_[0]],dtype = float) # List of hopping rates for ribosomes that can jump
        self.rib_pos_ = np.array([], dtype=int) # List of positions of ribosomes
        
        # Initialize variables of simulation
        t = 0
        no_rib_enter = 0
        no_rib_exit = 0
        runs = 0
        entry_current_new = 0
        self.a = self.w_[0]
        
        while True:
            
            #Run simulation step
            t, enter, do_exit, move =  self.__Sim_run(t)
            
            runs += 1
            
            #Update number of ribosomes entered and exited
            if enter:
                no_rib_enter += 1
            elif exit:
                no_rib_exit += 1
              
            if runs == 1:
                entry_current_new = no_rib_enter/t
            
         
            
            if self.RUNS == 0 and ((runs/self.LEN)%500) == 0:
               
                #Calculate currents
                entry_current_old = entry_current_new
                entry_current_new = no_rib_enter/t
                 
                # Check if Steady state reached if runs == 0
                if self.__checkSteadyState(entry_current_old,entry_current_new):
                
                    break
            
            # If runs != 0 then stop if number of runs reached
            elif self.RUNS !=0 and runs >=self.RUNS:
                
                break
            
            
        # If runs == 0, set runs of the sumulation as the runs it took to reach steady state   
        if self.RUNS == 0:
            
            self.RUNS = runs
            
    
    @jit
    def Run_Sim(self):
        
        """
        Run simulation after reaching steady state
                                
        """
        #Run simulation
        
        #Initialize simulation variables
        self.TIME = 0
        self.current = 0
        self.rho_ = np.zeros(self.LEN)
        
        no_rib_enter = 0
        
        
        # Run simulation
        for i in range(self.RUNS):
            
            self.TIME, enter, do_exit,move = self.__Sim_run(self.TIME,True)
            
            if enter:
                
                no_rib_enter += 1
            
        
        # Calculate current
        self.current = no_rib_enter/self.TIME
        # Calculate density profile
        self.rho_ = self.rho_/self.TIME
        self.rho_ = self.rho_[1:]
        

    @jit
    def __Sim_run(self, t = 0, steady_reached = False):
        
#         sim_list_ = [self.a,self.L,self.LEN,self.w_,self.rib_pos_,self.pos_jump_,self.pos_w_]
        
 
        
        sim_list_ = Sim_run_helper(self.a,self.L,self.LEN,
                                 self.w_,self.rib_pos_,self.pos_jump_,self.pos_w_,Get_tau,Get_mu)
                
#         print(here)
        self.a = sim_list_[0]    
        self.rib_pos_ = sim_list_[1]  
        self.pos_jump_ = sim_list_[2]  
        self.pos_w_ = sim_list_[3]  
        tau = sim_list_[4]
        enter = sim_list_[5]
        do_exit = sim_list_[6]
        move = sim_list_[7]
        
        t += tau
        
        if steady_reached:
            
            self.__calcDensity(tau)
        
        return t, enter, do_exit, move


    @jit
    def __checkSteadyState(self,entry_current_old,entry_current_new):
        
        if abs((entry_current_old - entry_current_new)*100/entry_current_old) < self.rel_tol:
            
            return True
        
        else: 
            
            return False
        
    @jit
    def __calcDensity(self,tau):
        
        for i in self.rib_pos_:
            
            self.rho_[i-1] += tau
        
        return
    @jit
    def getTot_den(self):
        
        return np.sum(self.rho_)/len(self.rho_)







In [5]:
sim = Simulation(10 , sweeps = 5000) 
file_name = '../'+'elongation-rates/'+'YBL108W_rates.dat'
# file_name = '../kvalue_dummy.dat'
sim.ExtractRates(file_name,0.1,18.03)

In [ ]:
# %%timeit
sim.Reach_steady()
sim.Run_Sim()